In [1]:
kernel.silent(true) // supress all non-explicit output
import ammonite.ops._
import sourcecode.Text.generate
val jarString = "../target/scala-3.0.2/euclid-scala-assembly-0.1.0-SNAPSHOT.jar"
val path = ammonite.ops.pwd / ammonite.ops.RelPath(jarString)
interp.load.cp(path)


import ammonite.ops._

import sourcecode.Text.generate

jarString: String = "../target/scala-3.0.2/euclid-scala-assembly-0.1.0-SNAPSHOT.jar"
path: Path = /Users/michaelmccoy/Projects/euclid-scala/target/scala-3.0.2/euclid-scala-assembly-0.1.0-SNAPSHOT.jar

In [2]:
def makeExponents(order: Int, l: List[Int]): List[(Int, Int)] = {
    l.zipWithIndex.map(t => (t._2, t._1)) :::((order, 1) :: Nil)
}

In [3]:
import com.mbmccoy.euclid.polynomial._
import scala.annotation.tailrec

val prime = Prime.next(65537)
val order = 1
val pf = PrimeField(prime)
println(pf)


com.mbmccoy.euclid.polynomial.PrimeField(65537)


In [12]:
def getOrder(x: pf.Element): Int = {
    var y = x
    var order = 1
    while (y != pf.one) {
        y *= x
        order += 1
    }
    order
}
    
// Print some orders here
var x = pf.one
for (_ <- 0 until 10) {
    val order = getOrder(x)
    println(f"x=${x}, order=${order}")
    x = x + pf.one
}

x=1, order=1
x=2, order=32
x=3, order=65536
x=4, order=16
x=5, order=65536
x=6, order=65536
x=7, order=65536
x=8, order=32
x=9, order=32768
x=10, order=65536


In [22]:
val omegaHat = pf(141) // order = 256
var omega = omegaHat.pow(64) // 4, order = 4
val omegaOrder = getOrder(omega)
val omegaHatOrder = getOrder(omegaHat)
def printOrder(x: pf.Element) = println(f"${x}: order=${getOrder(x)}")


/*for (i <- 1 to 100) {
    omega = omega * omegaHat
    println(f"${i}: ::${omega}:: order=${getOrder(omega)}")
}*/
printOrder(omegaHat)
printOrder(omega)


141: order=256
65281: order=4


In [23]:

val M = pf(44734)
val K1 = pf(34369)
val K2 = pf(51062)
val MInverse = M.reciprocal

// TODO: verify that this is close enough to a width-1 Rescue hash.
def rescueIteration(x: pf.Element): pf.Element = {
    val y: pf.Element = M * x.pow(3) + K1
    val cubeRoot = y.reciprocal.pow((pf.order.toInt - 2)/3)
    cubeRoot * MInverse + K2
}

def rescueHash(x: pf.Element, rounds: Int): pf.Element = {
    if (rounds <= 0) { 
        x
    } else {
        rescueHash(rescueIteration(x), rounds - 1)
    }
}


In [24]:
val fortyTwo = pf(42)

@tailrec
def winterfellReversed(x: pf.Element, rounds: Int, trace: List[pf.Element] = Nil): List[pf.Element] = {
    if (rounds <= 0) trace
    else {
        winterfellReversed(x*x*x + fortyTwo, rounds - 1, x :: trace)
    }
}

In [26]:
//val secret = pf(54234)
val secret = pf(1111)
val executionTrace = winterfellReversed(secret, 4).reverse
println(executionTrace)


List(1111, 34485, 15064, 25005)


In [27]:

def getSubgroup(x: pf.Element) = {
    lazy val subgroup: LazyList[pf.Element] = x #:: subgroup.map{y => x*y}
    pf.one :: subgroup.takeWhile(_ != pf.one).toList
}
val omegas = getSubgroup(omega)

println(omegas)
for (((omega, trace), i) <- omegas.zip(executionTrace).zipWithIndex ) {
    println(f"${i} | ${omega} | ${trace}")
}

List(1, 65281, 65536, 256)
0 | 1 | 1111
1 | 65281 | 34485
2 | 65536 | 15064
3 | 256 | 25005


In [28]:
import com.mbmccoy.euclid.polynomial.FieldOps.lagrangeInterpolation

val alpha = omegas.reduce((a,b) => (pf.one-a)*(pf.one-b))
val alphaInv = alpha.reciprocal
val rootPoly = Polynomial[pf.Element]((0, -pf.one), (omegaOrder, pf.one))
val interpolant = omegas.zip(executionTrace)
.map((w, y) => {
    val poly = rootPoly / (Polynomial[pf.Element]((0, -w), (1, pf.one)))
    y * poly / poly(w)
}).reduce((x, y) => x + y)

println(interpolant)
omegas.zip(executionTrace).map((w, y) => {
    println((interpolant(w), y))
})

2532 + 29783X + 38324X^2 + 61546X^3
(1111,1111)
(34485,34485)
(15064,15064)
(25005,25005)


In [35]:
val x = pf(134)
println(interpolant(omega*x) - interpolant(x).pow(3) - pf(42))

40460


In [11]:
println("x | f(x)")
println("--|-----")
getSubgroup(omegaHat).zipWithIndex.map((x, i) => {
    println(f"$$\hat{\omega}^{${i}}$$ | ${interpolant(x)}")
})

x | f(x)
--|-----
$\hat{\omega}^{0}$ | 1111
$\hat{\omega}^{1}$ | 61985
$\hat{\omega}^{2}$ | 0
$\hat{\omega}^{3}$ | 24359
$\hat{\omega}^{4}$ | 0
$\hat{\omega}^{5}$ | 43362
$\hat{\omega}^{6}$ | 0
$\hat{\omega}^{7}$ | 42777
$\hat{\omega}^{8}$ | 0
$\hat{\omega}^{9}$ | 32283
$\hat{\omega}^{10}$ | 0
$\hat{\omega}^{11}$ | 55295
$\hat{\omega}^{12}$ | 0
$\hat{\omega}^{13}$ | 36322
$\hat{\omega}^{14}$ | 0
$\hat{\omega}^{15}$ | 36490
$\hat{\omega}^{16}$ | 0
$\hat{\omega}^{17}$ | 12558
$\hat{\omega}^{18}$ | 0
$\hat{\omega}^{19}$ | 35031
$\hat{\omega}^{20}$ | 0
$\hat{\omega}^{21}$ | 15434
$\hat{\omega}^{22}$ | 0
$\hat{\omega}^{23}$ | 58915
$\hat{\omega}^{24}$ | 0
$\hat{\omega}^{25}$ | 23469
$\hat{\omega}^{26}$ | 0
$\hat{\omega}^{27}$ | 53072
$\hat{\omega}^{28}$ | 0
$\hat{\omega}^{29}$ | 18212
$\hat{\omega}^{30}$ | 59849
$\hat{\omega}^{31}$ | 46834
$\hat{\omega}^{32}$ | 0
$\hat{\omega}^{33}$ | 59072
$\hat{\omega}^{34}$ | 0
$\hat{\omega}^{35}$ | 5974
$\hat{\omega}^{36}$ | 0
$\hat{\omega}^{37}$ | 6291

x | f(x)
--|-----
$\hat{\omega}^{0}$ | 1111
$\hat{\omega}^{1}$ | 6760
$\hat{\omega}^{2}$ | 59183
$\hat{\omega}^{3}$ | 50423
$\hat{\omega}^{4}$ | 34485
$\hat{\omega}^{5}$ | 38024
$\hat{\omega}^{6}$ | 59117
$\hat{\omega}^{7}$ | 20665
$\hat{\omega}^{8}$ | 15064
$\hat{\omega}^{9}$ | 35863
$\hat{\omega}^{10}$ | 50630
$\hat{\omega}^{11}$ | 39013
$\hat{\omega}^{12}$ | 25005
$\hat{\omega}^{13}$ | 60555
$\hat{\omega}^{14}$ | 37809
$\hat{\omega}^{15}$ | 31101